In [118]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import ast
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords 

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/tmdb-movie-metadata/tmdb_5000_movies.csv
/kaggle/input/tmdb-movie-metadata/tmdb_5000_credits.csv


In [119]:
Credits=pd.read_csv("/kaggle/input/tmdb-movie-metadata/tmdb_5000_credits.csv")
Movies=pd.read_csv("/kaggle/input/tmdb-movie-metadata/tmdb_5000_movies.csv")

In [120]:
print(Credits.head())
print(Credits.shape)

   movie_id                                     title  \
0     19995                                    Avatar   
1       285  Pirates of the Caribbean: At World's End   
2    206647                                   Spectre   
3     49026                     The Dark Knight Rises   
4     49529                               John Carter   

                                                cast  \
0  [{"cast_id": 242, "character": "Jake Sully", "...   
1  [{"cast_id": 4, "character": "Captain Jack Spa...   
2  [{"cast_id": 1, "character": "James Bond", "cr...   
3  [{"cast_id": 2, "character": "Bruce Wayne / Ba...   
4  [{"cast_id": 5, "character": "John Carter", "c...   

                                                crew  
0  [{"credit_id": "52fe48009251416c750aca23", "de...  
1  [{"credit_id": "52fe4232c3a36847f800b579", "de...  
2  [{"credit_id": "54805967c3a36829b5002c41", "de...  
3  [{"credit_id": "52fe4781c3a36847f81398c3", "de...  
4  [{"credit_id": "52fe479ac3a36847f813eaa3",

In [121]:
print(Movies.head())
print(Movies.shape)

      budget                                             genres  \
0  237000000  [{"id": 28, "name": "Action"}, {"id": 12, "nam...   
1  300000000  [{"id": 12, "name": "Adventure"}, {"id": 14, "...   
2  245000000  [{"id": 28, "name": "Action"}, {"id": 12, "nam...   
3  250000000  [{"id": 28, "name": "Action"}, {"id": 80, "nam...   
4  260000000  [{"id": 28, "name": "Action"}, {"id": 12, "nam...   

                                       homepage      id  \
0                   http://www.avatarmovie.com/   19995   
1  http://disney.go.com/disneypictures/pirates/     285   
2   http://www.sonypictures.com/movies/spectre/  206647   
3            http://www.thedarkknightrises.com/   49026   
4          http://movies.disney.com/john-carter   49529   

                                            keywords original_language  \
0  [{"id": 1463, "name": "culture clash"}, {"id":...                en   
1  [{"id": 270, "name": "ocean"}, {"id": 726, "na...                en   
2  [{"id": 470, "nam

In [122]:
movies=Movies.merge(Credits, on='title')
print(movies.shape)
print(movies.info())

(4809, 23)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 4809 entries, 0 to 4808
Data columns (total 23 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4809 non-null   int64  
 1   genres                4809 non-null   object 
 2   homepage              1713 non-null   object 
 3   id                    4809 non-null   int64  
 4   keywords              4809 non-null   object 
 5   original_language     4809 non-null   object 
 6   original_title        4809 non-null   object 
 7   overview              4806 non-null   object 
 8   popularity            4809 non-null   float64
 9   production_companies  4809 non-null   object 
 10  production_countries  4809 non-null   object 
 11  release_date          4808 non-null   object 
 12  revenue               4809 non-null   int64  
 13  runtime               4807 non-null   float64
 14  spoken_languages      4809 non-null   object 
 15  status    

In [123]:
movies=movies[['movie_id','title','genres','overview','keywords','cast','crew']]

In [124]:
print(movies.iloc[0])

movie_id                                                19995
title                                                  Avatar
genres      [{"id": 28, "name": "Action"}, {"id": 12, "nam...
overview    In the 22nd century, a paraplegic Marine is di...
keywords    [{"id": 1463, "name": "culture clash"}, {"id":...
cast        [{"cast_id": 242, "character": "Jake Sully", "...
crew        [{"credit_id": "52fe48009251416c750aca23", "de...
Name: 0, dtype: object


***Here genres are in string format but for our objective we only need the name of geners like action , adventure in form of a list (same goes for others)***

In [125]:
movies['genres'][0]

'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

In [126]:
#to remove the missing data if something is not present as well as removing duplicate data if present
print(movies.isnull().sum())
print(movies.duplicated().sum())

movie_id    0
title       0
genres      0
overview    3
keywords    0
cast        0
crew        0
dtype: int64
0


In [127]:
#no duplicate data so just removing those rows whose overfiew is not there
movies.dropna(inplace=True)

In [128]:
#coverting string into a list
def extract(z):
    l=[]
    for j in ast.literal_eval(z):
        l.append(j["name"])
    return l

#for taking only top 4 cast
def extcast(z):
    l=[]
    c=0
    for i in ast.literal_eval(z):
        l.append(i['name'])
        c+=1
        if c==4:
            break
    return l  

#only taking Director
def extcrew(z):
    l=[]
    for i in ast.literal_eval(z):
        if i['job']=='Director':
            l.append(i['name'])
            break
    return l

In [129]:
movies['genres']=movies['genres'].apply(extract)
movies['keywords']=movies['keywords'].apply(extract)
movies['cast']=movies['cast'].apply(extcast)
movies['crew']=movies['crew'].apply(extcrew)
movies['overview']=movies['overview'].apply(lambda x:x.split())

For Removing Spaces

In [130]:
movies['genres']=movies['genres'].apply(lambda x:[j.replace(" ","") for j in x])
movies['keywords']=movies['keywords'].apply(lambda x:[j.replace(" ","") for j in x])
movies['cast']=movies['cast'].apply(lambda x:[j.replace(" ","") for j in x])
movies['crew']=movies['crew'].apply(lambda x:[j.replace(" ","") for j in x])

In [131]:
movies.head()

,movie_id,title,genres,overview,keywords,cast,crew
0,19995,Avatar,"[Action, Adventure, Fantasy, ScienceFiction]","[In, the, 22nd, century,, a, paraplegic, Marin...","[cultureclash, future, spacewar, spacecolony, ...","[SamWorthington, ZoeSaldana, SigourneyWeaver, ...",[JamesCameron]
1,285,Pirates of the Caribbean: At World's End,"[Adventure, Fantasy, Action]","[Captain, Barbossa,, long, believed, to, be, d...","[ocean, drugabuse, exoticisland, eastindiatrad...","[JohnnyDepp, OrlandoBloom, KeiraKnightley, Ste...",[GoreVerbinski]
2,206647,Spectre,"[Action, Adventure, Crime]","[A, cryptic, message, from, Bond’s, past, send...","[spy, basedonnovel, secretagent, sequel, mi6, ...","[DanielCraig, ChristophWaltz, LéaSeydoux, Ralp...",[SamMendes]
3,49026,The Dark Knight Rises,"[Action, Crime, Drama, Thriller]","[Following, the, death, of, District, Attorney...","[dccomics, crimefighter, terrorist, secretiden...","[ChristianBale, MichaelCaine, GaryOldman, Anne...",[ChristopherNolan]
4,49529,John Carter,"[Action, Adventure, ScienceFiction]","[John, Carter, is, a, war-weary,, former, mili...","[basedonnovel, mars, medallion, spacetravel, p...","[TaylorKitsch, LynnCollins, SamanthaMorton, Wi...",[AndrewStanton]


***MEARGING ALL THE IMP COLUMNS TO FORM A GROUP***

In [132]:
movies['total']=movies['overview']+movies['genres']+movies['keywords']+movies['cast']+movies['crew']

In [133]:
new_movies=movies[['movie_id','title','total']]

In [134]:
new_movies.head()

,movie_id,title,total
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin..."
1,285,Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d..."
2,206647,Spectre,"[A, cryptic, message, from, Bond’s, past, send..."
3,49026,The Dark Knight Rises,"[Following, the, death, of, District, Attorney..."
4,49529,John Carter,"[John, Carter, is, a, war-weary,, former, mili..."


In [135]:
#FOR MAKING SLIGHTLY SAME WORDS TO A SAME WORD
ps=PorterStemmer()
def st(x):
    y=[]
    for i in x.split():
        y.append(ps.stem(i))
    return " ".join(y)

In [ ]:
new_movies['total']=new_movies['total'].apply(lambda x:" ".join(x))
new_movies['total']=new_movies['total'].apply(lambda x:x.lower())
new_movies['total']=new_movies['total'].apply(st)

In [137]:
Cv=CountVectorizer(max_features=5200,stop_words='english')
v=Cv.fit_transform(new_movies['total']).toarray()
m,n=v.shape

In [140]:
new_movies['total'][0]

'in the 22nd century, a parapleg marin is dispatch to the moon pandora on a uniqu mission, but becom torn between follow order and protect an alien civilization. action adventur fantasi sciencefict cultureclash futur spacewar spacecoloni societi spacetravel futurist romanc space alien tribe alienplanet cgi marin soldier battl loveaffair antiwar powerrel mindandsoul 3d samworthington zoesaldana sigourneyweav stephenlang jamescameron'

In [141]:
similarity=np.zeros((m,m))
n1=np.linalg.norm(v,axis=1,keepdims=True)
d=np.dot(v,v.T)
d1=np.dot(n1,n1.T)
similarity=(d/d1)
print(similarity.shape)

(4806, 4806)


In [142]:
sorted(list(enumerate(similarity[0])),reverse=True,key=lambda x:x[1])[1:8]

[(1216, 0.28676966733820225),
 (2409, 0.26310068027921696),
 (507, 0.25560859370538297),
 (3730, 0.2539166875385041),
 (539, 0.24678382369818683),
 (582, 0.24511108480187255),
 (1204, 0.23918243661746996)]

In [143]:
def recommend(movie):
    movie_i=new_movies[new_movies['title']==movie].index[0]
    m_list=sorted(list(enumerate(similarity[:,movie_i])),reverse=True,key=lambda x:x[1])[1:8]
    print("YOU SHOULD GO FOR THIS MOVIES NEXT -----> 🎬")
    for i in m_list:
        print(new_movies.iloc[i[0]].title)


In [148]:
movie=input("Enter movie name (should be accuracte)🎥: ")
recommend(movie)
#Harry Potter and the Order of the Phoenix
#Harry Potter and the Chamber of Secrets
#Harry Potter and the Half-Blood Prince

Enter movie name (should be accuracte)🎥:  Superman Returns


YOU SHOULD GO FOR THIS MOVIES NEXT -----> 🎬
Superman II
Superman III
Superman IV: The Quest for Peace
Superman
The Wolverine
Iron Man 2
The Beastmaster
